# Encode embeddings and index documents with IDs

In [1]:
# load created document with ids

import json 

with open ('documents-with-ids.json', 'rt') as f_in: 
    documents = json.load(f_in)

In [2]:
from sentence_transformers import SentenceTransformer

In [4]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [6]:
# example to test
v = model.encode('I just discovered the course. Can I still join?')
v.shape

(384,)

In [7]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [8]:
# compute embeddings 

from tqdm import tqdm

for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [09:55<00:00,  1.59it/s]


In [9]:
# index documents 

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:09<00:00, 102.85it/s]


# Search query

In [10]:
# encode querry 

query = 'I just discovered the course. Can I still join it?'
v_q = model.encode(query)

In [11]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [13]:
import pandas as pd

df_ground_truth = pd.read_csv('ground-truth-data.csv')
ground_truth = df_ground_truth.to_dict(orient='records')
ground_truth[0]


{'question': 'What is the start date and time for the course?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

In [14]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [15]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

# Evaluate question embeddings 

In [12]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

In [16]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [17]:
evaluate(ground_truth, question_vector_knn)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4735/4735 [08:09<00:00,  9.68it/s]


{'hit_rate': 0.7408658922914467, 'mrr': 0.6264378739880324}

# Evaluate answer embeddings 

In [20]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('text_vector', v_q, course)

In [21]:
evaluate(ground_truth, text_vector_knn)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4735/4735 [07:59<00:00,  9.88it/s]


{'hit_rate': 0.8359028511087645, 'mrr': 0.6980394227384731}

In [25]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

In [26]:
evaluate(ground_truth, question_text_vector_knn)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4735/4735 [07:46<00:00, 10.15it/s]


{'hit_rate': 0.9191129883843717, 'mrr': 0.8027771911298851}

In [27]:
def elastic_search_knn_combined(vector, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "course": course
                                }
                            },
                            # script to compute cosine similarity 
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'text_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_text_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [28]:
def vector_combined_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, course)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4735/4735 [07:52<00:00, 10.01it/s]


{'hit_rate': 0.9009503695881732, 'mrr': 0.7839985920450552}

In [29]:
evaluate(ground_truth, vector_combined_knn)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 4735/4735 [07:54<00:00,  9.97it/s]


{'hit_rate': 0.9009503695881732, 'mrr': 0.7839985920450552}